# Agent 작성

주제 - 내가 배운 모든 걸 알고 있는 학습 조교

data - TIL 안의 모든 md 파일

Tools: RAG, 웹서치



In [29]:
# base
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# agent
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# memory
from langchain.memory import ConversationBufferMemory

# web search
from langchain_tavily import TavilySearch

# RAG
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import MarkdownTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.tools.retriever import create_retriever_tool

# 출력파서
from pydantic import BaseModel, Field

load_dotenv()

True

## 기초
### LLM

In [6]:
# LLM
llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)

### 프롬프트

In [ ]:
# 프롬프트

system_message = f'''
너는 웹 검색도 가능하고, 사용자가 AI 기반 데이터 분석가 양성 과정에서 공부한 내용도 검색할 수 있는 어시스턴트야.



사용자가 일반적인 질문을 하거나, 최신성 혹은 팩트 체크가 필요하자 않다면 도구를 사용하지 않아도 좋아.

만약 답을 모르겠다면, 그냥 답을 모르겠다고 말해.

가장 의미있는 검색결과들을 요약해서 질문에 답해줘.

'''


prompt = ChatPromptTemplate([
    ('system', system_message),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')
])


### 메모리

In [8]:
# 메모리
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key='chat_history'
)

## Tools
### 웹서치

In [31]:
# 웹서치
search_tool = TavilySearch(
    max_results=5,
    topic='general',
    search_depth='advanced'
)

### RAG

In [ ]:
# 사전처리
def load_markdown(folder_path):
    loader = DirectoryLoader(
        folder_path,
        glob='**/*.md',
        loader_cls=lambda path: TextLoader(path, encoding="utf-8"),
        show_progress=True
    )
    docs = loader.load()
    return docs

folder_path = '../'
documents = load_markdown(folder_path)

text_splitter = MarkdownTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

split_docs = text_splitter.split_documents(documents)

embedding = OpenAIEmbeddings()

vectorstore = Chroma.from_documents(documents=split_docs, embedding=embedding)

100%|██████████| 47/47 [00:00<00:00, 5331.50it/s]


In [30]:
# RAG
retriever = vectorstore.as_retriever()

rag_tool = create_retriever_tool(
    retriever,
    name='md_search',
    description='수업자료에서 관련된 내용을 검색합니다'
)

In [ ]:
# Tools 설정
tools = [search_tool, rag_tool]

## 조합

In [ ]:
agent = create_openai_tools_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True    
)

agent_executor.invoke({'input': ''})